In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
from os import listdir
from os.path import isfile, join
from datetime import time
import glob
import sys
import datetime

from pyspark.sql.functions import udf, hour, mean, minute, stddev, count, max as psmax, min as psmin, date_format

from pyspark.sql import SQLContext
from pyspark.sql.types import *

Go here to see pyspark functions
http://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html

In [ ]:
#data locations
data_5min_path_d11 = "../../../d11_traffic_data/d11/"
data_5min_path_d8 = "../../../d8_traffic_data/"
meta_path_d11 = "../../../d11_traffic_data/meta/d11/"

# Get data file names

These are all of the filenames that are going to be fed into the Spark instance.

In [ ]:
#get all files to process
#onlyfiles = [f for f in listdir(data_5min_path) if isfile(join(data_5min_path, f)) ]
#onlyfiles = [a for a in onlyfiles if 'txt.gz' in a]
onlyfiles = [data_5min_path_d11 + a for a in listdir(data_5min_path_d11) if 'txt.gz' in a ] + \
            [data_5min_path_d8 + b for b in listdir(data_5min_path_d8) if 'txt.gz' in b ] 
onlyfiles

# Make spark schema

struct list was generated with the following code after reading the files with inferschema = true, then hand modified 
```
'[' + ','.join(['StructField("%s",%s(),True)'% (colnames[idx], str(i.dataType))
for idx, i in enumerate(rdd.schema)]) + ']'
```

 Column Names are not defined in the files. Defining column names for readability

In [ ]:
colnames = [
    'Timestamp','Station','District','Freeway','Direction_of_Travel',
    'LaneType','StationLength','Samples',
    'Perc_Observed','TotalFlow','AvgOccupancy','AvgSpeed',
    'Lane1_Samples','Lane1_Flow','Lane1_AvgOcc','Lane1_AvgSpeed','Lane1_Observed',
    'Lane2_Samples','Lane2_Flow','Lane2_AvgOcc','Lane2_AvgSpeed','Lane2_Observed',
    'Lane3_Samples','Lane3_Flow','Lane3_AvgOcc','Lane3_AvgSpeed','Lane3_Observed',
    'Lane4_Samples','Lane4_Flow','Lane4_AvgOcc','Lane4_AvgSpeed','Lane4_Observed',
    'Lane5_Samples','Lane5_Flow','Lane5_AvgOcc','Lane5_AvgSpeed','Lane5_Observed',
    'Lane6_Samples','Lane6_Flow','Lane6_AvgOcc','Lane6_AvgSpeed','Lane6_Observed',
    'Lane7_Samples','Lane7_Flow','Lane7_AvgOcc','Lane7_AvgSpeed','Lane7_Observed',
    'Lane8_Samples','Lane8_Flow','Lane8_AvgOcc','Lane8_AvgSpeed','Lane8_Observed'
]
colnames = [c.lower() for c in colnames]

Ran spark instance once with limited dataset to identify the datatypes of each column to help create the following script.

In [ ]:
#print '[\n    ' + ",\n    ".join(['StructField("%s",%s(),True)'% (colnames[idx], str(i.dataType))
#for idx, i in enumerate(rdd.schema)]) + '\n]'

# Build dataframe with spark

Defining the schema of the files that we are reading in. StructType creates the dataframe schema equivalent of create syntax in SQL)

In [ ]:
struct_list = [
    StructField("timestamp",TimestampType(),True),
    StructField("station",IntegerType(),True),
    StructField("district",IntegerType(),True),
    StructField("freeway",IntegerType(),True),
    StructField("direction_of_travel",StringType(),True),
    StructField("lanetype",StringType(),True),
    StructField("stationlength",DoubleType(),True),
    StructField("samples",IntegerType(),True),
    StructField("perc_observed",IntegerType(),True),
    StructField("totalflow",IntegerType(),True),
    StructField("avgoccupancy",DoubleType(),True),
    StructField("avgspeed",DoubleType(),True),
    StructField("lane1_samples",IntegerType(),True),
    StructField("lane1_flow",IntegerType(),True),
    StructField("lane1_avgocc",DoubleType(),True),
    StructField("lane1_avgspeed",DoubleType(),True),
    StructField("lane1_observed",IntegerType(),True),
    StructField("lane2_samples",IntegerType(),True),
    StructField("lane2_flow",IntegerType(),True),
    StructField("lane2_avgocc",DoubleType(),True),
    StructField("lane2_avgspeed",DoubleType(),True),
    StructField("lane2_observed",IntegerType(),True),
    StructField("lane3_samples",IntegerType(),True),
    StructField("lane3_flow",IntegerType(),True),
    StructField("lane3_avgocc",DoubleType(),True),
    StructField("lane3_avgspeed",DoubleType(),True),
    StructField("lane3_observed",IntegerType(),True),
    StructField("lane4_samples",IntegerType(),True),
    StructField("lane4_flow",IntegerType(),True),
    StructField("lane4_avgocc",DoubleType(),True),
    StructField("lane4_avgspeed",DoubleType(),True),
    StructField("lane4_observed",IntegerType(),True),
    StructField("lane5_samples",IntegerType(),True),
    StructField("lane5_flow",IntegerType(),True),
    StructField("lane5_avgocc",DoubleType(),True),
    StructField("lane5_avgspeed",DoubleType(),True),
    StructField("lane5_observed",IntegerType(),True),
    StructField("lane6_samples",IntegerType(),True),
    StructField("lane6_flow",IntegerType(),True),
    StructField("lane6_avgocc",DoubleType(),True),
    StructField("lane6_avgspeed",DoubleType(),True),
    StructField("lane6_observed",IntegerType(),True),
    StructField("lane7_samples",IntegerType(),True),
    StructField("lane7_flow",IntegerType(),True),
    StructField("lane7_avgocc",DoubleType(),True),
    StructField("lane7_avgspeed",DoubleType(),True),
    StructField("lane7_observed",IntegerType(),True),
    StructField("lane8_samples",IntegerType(),True),
    StructField("lane8_flow",IntegerType(),True),
    StructField("lane8_avgocc",DoubleType(),True),
    StructField("lane8_avgspeed",DoubleType(),True),
    StructField("lane8_observed",IntegerType(),True)
]

schema_struct = StructType(struct_list)

Loading the data into spark dataframe from the files (equivalent of insert statements with files in SQL)

In [ ]:
limit_files = onlyfiles[:5] + onlyfiles[-5:]
limit_files

In [ ]:
#node this is only the first 5 days of files for now
limit_files = onlyfiles[:5] + onlyfiles[-5:]
files = [filename for filename in limit_files]

rdd = spark.read.csv(
    files, 
    header='false',
    timestampFormat='MM/dd/yyyy HH:mm:ss',
    schema=schema_struct,
    inferSchema='false'
)
    
rdd.take(1)

In [ ]:
rdd.count()

### Build freeway STM station order from meta data

Build a sparksql dataframe with the metadata

In [ ]:
meta_path = "../../../d11_traffic_data/meta/d11/"

def loadMeta():
    meta_dir= meta_path+'d11_text_meta_2015_*.txt'
    meta_files = glob.glob(meta_dir)

    meta_file_list = []
    for meta_file in meta_files:
        date = str('_'.join(meta_file.split('_')[4:7])).split('.')[0]
        df = pd.read_table(meta_file, index_col=None, header=0)
        date_col = pd.Series([date] * len(df))
        df['file_date'] = date_col
        # drop rows that are missing latitude / longitude values
        #df.dropna(inplace=True, subset=['Latitude', 'Longitude'], how='any')
        meta_file_list.append(df)

    meta_frame = pd.concat(meta_file_list).drop_duplicates(subset='ID', keep='last')

    usefwy = [ 56, 125, 805,  52, 163,   8,  15,   5, 905,  78,  94,  54]

    meta_frame = meta_frame[meta_frame.Fwy.apply(lambda x: x in usefwy)]

    #Add freeway name FwyDir
    meta_frame['freeway'] = meta_frame.Fwy.apply(str) + meta_frame.Dir
    
    r_c = {}
    for c in meta_frame.columns:
        r_c[c]=c.lower()
    
    meta_frame=meta_frame.rename(columns = r_c )
    return meta_frame

meta_data = sqlCtx.createDataFrame(loadMeta().loc[:,['id','abs_pm','type']].rename(columns={'id':'station'}))

In [ ]:
meta_data.show(100)

# filter for weekdays I5 S
# group by station, time


Modify this to make all queries

In [ ]:
weekdaySelector = udf(
    lambda x: "weekday" if int(x) < 6 else "weekend"
)

timeOfDay = udf(
    lambda x: time(int(x.hour), int(x.minute)).strftime("%H:%M")
)


In [ ]:
station_time = (
    rdd
    .select(
        'district',
        'freeway',
        'direction_of_travel',
        'timestamp',
        'station',
        'totalflow',
        'avgoccupancy',
        'avgspeed',
        date_format('timestamp', 'u').alias('dayofweek')
    )
    
)

station_time = (
    station_time
    .withColumn(
        'dayType', 
        weekdaySelector(station_time.dayofweek)
        )
    .withColumn(
        'timeOfDay',
        timeOfDay(station_time.timestamp)
    )
    .groupBy([
        'district',
        'freeway',
        'direction_of_travel',
        'station',
        'dayType',
        'timeOfDay'
    ])
    .agg(
        mean("totalflow").alias("flow_mean"),
        stddev("totalflow").alias("flow_std"),
        count("totalflow").alias("flow_count"),
        psmax("totalflow").alias("flow_max"),
        psmin("totalflow").alias("flow_min"),
        
        mean("avgoccupancy").alias("occ_mean"),
        stddev("avgoccupancy").alias("occ_std"),
        count("avgoccupancy").alias("occ_count"),
        psmax("avgoccupancy").alias("occ_max"),
        psmin("avgoccupancy").alias("occ_min"),
        
        mean("avgspeed").alias("speed_mean"),
        stddev("avgspeed").alias("speed_std"),
        count("avgspeed").alias("speed_count"),
        psmax("avgspeed").alias("speed_max"),
        psmin("avgspeed").alias("speed_min")
    )
)


In [ ]:
df = station_time.toPandas()

In [ ]:
df.sort_values('timeOfDay')

In [ ]:
df.columns

In [ ]:
def reduce_data_by_dict(df, keyval_dict):
    for key, val in keyval_dict.iteritems():
        df = df[df[key] == val]
    return df

In [ ]:
%run ../trafficpassion/AnalyzeWiggles.py

In [ ]:
restrict_dict = {'dayType': 'weekday', 
                 'district': 11,
                 'freeway': 5,
                 'direction_of_travel': 'S'
                }

In [ ]:
reduce_data_by_dict(df,restrict_dict ).sort_values(['timeOfDay'])